In [1]:
!pip install unsloth

In [2]:
!pip install "unsloth @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers==0.0.27" trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-9sid1qh3/unsloth_14e7ce6ddd2345e2b014b7b82937b2cc
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-9sid1qh3/unsloth_14e7ce6ddd2345e2b014b7b82937b2cc
  Resolved https://github.com/unslothai/unsloth.git to commit 6ac4e2e36f2f8bd0bc63a6eb85afa7097948ff3d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!pip uninstall xformers -y
!pip3 install -U xformers --index-url https://download.pytorch.org/whl/cu126

Found existing installation: xformers 0.0.27
Uninstalling xformers-0.0.27:
  Successfully uninstalled xformers-0.0.27
Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 76.3 MB/s eta 0:00:00a 0:00:01


In [4]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096
dtype = None
load_in_4bit = False

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-07-09 10:43:57.611007: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-09 10:43:57.622353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752038037.635738 3459793 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752038037.639590 3459793 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752038037.650113 3459793 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "microsoft/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = torch.bfloat16,
    device_map = {"": torch.cuda.current_device()}
)

==((====))==  Unsloth 2025.7.1: Fast Llama patching. Transformers: 4.53.1.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.19 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj","v_proj", "o_proj", "gate_proj","up_proj","down_proj"],
    lora_alpha = 32,
    lora_dropout = 0.1,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.7.1 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [7]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="converted_train_merged.jsonl", split="train") ###replace with your training_data

In [81]:
class_weights = {
    " A": 0.4,
    " B": 0.4,
    " C": 0.4,
    " D": 0.5
}

def compute_sample_weight(example):
    weight = 1.0  # default weight

    try:
        answer = example["conversations"][-1]["value"].strip()
        token = f" {answer}"  # prepend space to match tokenizer output
        weight = class_weights.get(token, 1.0)
    except Exception as e:
        print(f"[WARN] Could not extract answer from: {example['conversations']}")
        # Keep default weight

    example["weight"] = weight
    return example

In [82]:
train_dataset = dataset.map(compute_sample_weight)

Map:   0%|          | 0/164702 [00:00<?, ? examples/s]

In [69]:
train_dataset[0]

{'id': '86b409a8-9790-48a7-b90f-ccf60a7417b9',
 'conversations': [{'from': 'human',
   'value': 'Question: Most common chronic ahritis seen in children ?\n\nOptions:\nA. JRA\nB. Rheumatic ahritis\nC. Rheumatic fever\nD. Sepsis\n\nAnswer with the correct option letter only.'},
  {'from': 'gpt', 'value': 'A'}],
 'weight': 0.4}

In [83]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3",
    mapping = {
        "role": "from",
        "content": "value",
        "user": "human",
        "assistant": "gpt"

    }
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    
    # Generate plain text from chat template
    texts = [
        tokenizer.apply_chat_template(
            convo, tokenize=False, add_generation_prompt=False
        ) for convo in convos
    ]
    
    # Tokenize the text batch
    tokenized = tokenizer(texts, truncation=True)

    # Pass along weights (should be a list of floats with same length as batch)
    tokenized["weight"] = examples["weight"]

    return tokenized

pass

In [84]:
mapped_dataset = train_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/164702 [00:00<?, ? examples/s]

In [72]:
mapped_dataset[0]

{'id': '86b409a8-9790-48a7-b90f-ccf60a7417b9',
 'conversations': [{'from': 'human',
   'value': 'Question: Most common chronic ahritis seen in children ?\n\nOptions:\nA. JRA\nB. Rheumatic ahritis\nC. Rheumatic fever\nD. Sepsis\n\nAnswer with the correct option letter only.'},
  {'from': 'gpt', 'value': 'A'}],
 'weight': 0.4,
 'input_ids': [32010,
  894,
  29901,
  7849,
  3619,
  17168,
  293,
  21023,
  768,
  275,
  3595,
  297,
  4344,
  1577,
  13,
  13,
  5856,
  29901,
  13,
  29909,
  29889,
  435,
  4717,
  13,
  29933,
  29889,
  390,
  354,
  398,
  2454,
  21023,
  768,
  275,
  13,
  29907,
  29889,
  390,
  354,
  398,
  2454,
  1238,
  369,
  13,
  29928,
  29889,
  922,
  567,
  275,
  13,
  13,
  22550,
  411,
  278,
  1959,
  2984,
  5497,
  871,
  29889,
  32007,
  32001,
  319,
  32007],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,

In [73]:
unsloth_template = """
{{ bos_token }}
You are a medical assistant answering multiple choice questions.\n
{% for message in messages %}
    {% if message['from'] == 'human' %}
        {{ '### Question:\n' + message['value'] + '\n' }}
    {% else %}
        {{ '### Answer: ' + message['value'] + eos_token + '\n' }}
    {% endif %}
{% endfor %}
"""
unsloth_eos_token = "eos_token"

if False:
    tokenizer = get_chat_template(
    tokenizer,
    chat_template = unsloth_template,
    eos_token = unsloth_eos_token,
    mapping = {
        "role": "from",
        "content": "value",
        "user": "human",
        "assistant": "gpt"
    },
    map_eos_token = True
    )

In [74]:
mapped_dataset[1]

{'id': 'train-09947',
 'conversations': [{'from': 'human',
   'value': 'Question: A 15-year-old boy presents with sudden onset right sided weakness of his arm and face and difficulty speaking. He denies any problems with hearing or comprehension. The patient has no history of chest pain, hypertension, or diabetes mellitus. No significant past medical history. The patient is afebrile, and vital signs are within normal limits. On physical examination, the patient is thin, with long arms and slender fingers. There is a right-sided facial droop present. Ophthalmic examination reveals a dislocated lens in the right eye. Strength is 3 out of 5 in the right upper extremity, and there is a positive Babinski reflex on the right. The CT scan of the head shows no evidence of hemorrhage. Laboratory findings are significant for increased concentrations of a metabolic intermediate in his serum and urine. Which of the following enzymes is most likely deficient in this patient?\n\nOptions:\nA. Phenyla

In [16]:
print(mapped_dataset[0].keys())  

dict_keys(['id', 'conversations', 'weight', 'input_ids', 'attention_mask'])


In [17]:
print(mapped_dataset[6]["weight"])

0.2054


In [85]:
from transformers import DataCollatorForLanguageModeling

mapped_dataset.set_format("torch", columns=["input_ids", "attention_mask", "weight"])

class DataCollatorWithWeights(DataCollatorForLanguageModeling):
    def __init__(self, tokenizer):
        super().__init__(tokenizer=tokenizer, mlm=False)
        
def __call__(self, examples):
    batch = super().__call__(examples)
    weights = []
    for ex in examples:
        weights.append(ex["weight"])
    batch["weight"] = torch.tensor(weights, dtype=torch.float)
    return batch



In [86]:
from trl import SFTTrainer
import torch.nn.functional as F

class SimpleWeightedSFT(SFTTrainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights  # tensor of size [vocab_size] or [num_classes]

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        inputs["labels"] = inputs["input_ids"].clone()
        labels = inputs["labels"]
        weights = inputs.data.get("weight")
        outputs = model(**inputs)
        logits = outputs.logits

        # Shift tokens for LM loss
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        attention_mask = inputs["attention_mask"][..., 1:].contiguous()

        # Raw loss (no reduction)
        token_loss = F.cross_entropy(
            shift_logits.view(-1, shift_logits.size(-1)),
            shift_labels.view(-1),
            reduction='none'
        ).view(shift_labels.size())

              # Mask out padding
        token_loss = token_loss * attention_mask
        
        # Sum per sequence
        loss_per_sample = token_loss.sum(dim=1) / attention_mask.sum(dim=1)
        
        # Apply per-sample weight here
        if "weight" in inputs:
            sample_weights = inputs["weight"].to(loss_per_sample.device)
            loss_per_sample = loss_per_sample * sample_weights
        
        # Final loss
        final_loss = loss_per_sample.mean()
        
        return (final_loss, outputs) if return_outputs else final_loss

In [89]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SimpleWeightedSFT(
    model = model,
    tokenizer = tokenizer,
    train_dataset= mapped_dataset,
    data_collator=DataCollatorWithWeights(tokenizer=tokenizer),
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,

    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [90]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 164,702 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416 of 3,850,963,968 (0.78% trained)


Step,Training Loss
1,0.717000
2,0.572000
3,0.675700
4,1.180000
5,0.729400
6,0.434300
7,0.445800
8,0.870200
9,0.596900
10,0.664500


In [49]:
model.save_pretrained("lora_model_v5") # Local saving
tokenizer.save_pretrained("lora_model_v5")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model_v5/tokenizer_config.json',
 'lora_model_v5/special_tokens_map.json',
 'lora_model_v5/chat_template.jinja',
 'lora_model_v5/tokenizer.model',
 'lora_model_v5/added_tokens.json',
 'lora_model_v5/tokenizer.json')

In [50]:
import json
paths = ["test_easy.jsonl", "test_medium.jsonl", "test_hard.jsonl"]

combined_test_data = []
for path in paths:
    with open(path, 'r') as f:
        for line in f:
            combined_test_data.append(json.loads(line))

print(f"Total combined examples: {len(combined_test_data)}")


Total combined examples: 300


In [51]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {
        "from": "human",
        "value": (
            "Generally accepted indications for mechanical ventilatory support include\n\n"
            "A. PaO2 of less than 70 kPa and PaCO2 of greater than 50 kPa while breathing room air\n"
            "B. Alveolar-arterial oxygen tension difference of 150 kPa while breathing 100% O2\n"
            "C. Vital capacity of 40-60 mL/kg\n"
            "D. Respiratory rate greater than 35 breaths/min\n\n"
            "Answer with the correct option (A/B/C/D) only."
        )
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['Generally accepted indications for mechanical ventilatory support include\n\nA. PaO2 of less than 70 kPa and PaCO2 of greater than 50 kPa while breathing room air\nB. Alveolar-arterial oxygen tension difference of 150 kPa while breathing 100% O2\nC. Vital capacity of 40-60 mL/kg\nD. Respiratory rate greater than 35 breaths/min\n\nAnswer with the correct option (A/B/C/D) only. A']

In [91]:
from unsloth.chat_templates import get_chat_template
from tqdm import tqdm
import pandas as pd

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
def format_question(item):
    q = item["question"]
    opts = item["options"]
    opt_str = "\n".join([f"{k}. {opts[k]}" for k in sorted(opts)])
    return (
        f"{q}\n\n"
        f"{opt_str}\n\n"
        "Answer with the correct option (A/B/C/D) only."
    )

def get_answer(messages):
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")
    outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=1,         # Only want one token (A/B/C/D)
            do_sample=False,
            use_cache=True,
        )
    generated_ids = outputs[0][input_ids.shape[1]:]  # Get only new tokens
    decoded = tokenizer.decode(generated_ids, skip_special_tokens=True).strip().upper()

    if decoded in [chr(c) for c in range(ord("A"), ord("Z") + 1)]:
        return decoded
    else: None

# Run through test data
results = []
for item in tqdm(combined_test_data):
    prompt = format_question(item)
    messages = [{"from": "human", "value": prompt}]
    answer = get_answer(messages)
    results.append({"id": item["id"], "answer": answer})

# Save to CSV
df = pd.DataFrame(results)
df.to_csv("submission_5.csv", index=False)
print("✅ Saved to submission.csv")

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:20<00:00, 14.73it/s]

✅ Saved to submission.csv
